In [6]:
import pandas as pd
import geopandas as gpd

## Load parks

In [3]:
df_parks = gpd.read_file("https://data.cityofnewyork.us/resource/enfh-gkve.geojson?$limit=1000000")

In [4]:
df_parks[df_parks['communityboard']=='103'].to_file('data/parks.geojson')

## Load subsidized housing

In [10]:
#df_subsidized = pd.read_csv("https://furmancenter.org/files/CoreData/FC_SHD_bbl_analysis_2024-08-29.csv")
df_subsidized = pd.read_csv("data/FC_SHD_bbl_analysis_2024-08-29.csv")
df_subsidized.columns[30:]

/var/folders/_q/g5l2s5950hz1zr9syt43gzp80000gn/T/ipykernel_73686/3682638536.py:2: DtypeWarning: Columns (103,106,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df_subsidized = pd.read_csv("data/FC_SHD_bbl_analysis_2024-08-29.csv")


Index(['data_dof', 'data_hcrlihtc', 'data_hpd', 'data_hudcon', 'data_hudfin',
       'data_hudlihtc', 'data_ml', 'data_nycha', 'sub_fin', 'sub_prog',
       'sub_tax', 'sub_zone', 'prog_202_8', 'start_202_8', 'end_202_8',
       'prog_221d', 'start_221d', 'end_221d', 'prog_223', 'start_223',
       'end_223', 'prog_420c', 'start_420c', 'end_420c', 'prog_421a',
       'start_421a', 'end_421a', 'prog_421a_aff', 'start_421a_aff',
       'end_421a_aff', 'prog_8a', 'start_8a', 'end_8a', 'prog_hpd_oth',
       'start_hpd_oth', 'end_hpd_oth', 'prog_hud_fin_oth', 'start_hud_fin_oth',
       'end_hud_fin_oth', 'prog_hud_proj_oth', 'start_hud_proj_oth',
       'end_hud_proj_oth', 'prog_inclus_hous', 'start_inclus_hous',
       'end_inclus_hous', 'prog_j51', 'start_j51', 'end_j51', 'prog_lamp',
       'start_lamp', 'end_lamp', 'prog_lihtc4', 'start_lihtc4', 'end_lihtc4',
       'prog_lihtc9', 'start_lihtc9', 'end_lihtc9', 'prog_lmsa', 'start_lmsa',
       'end_lmsa', 'prog_mfp', 'start_mfp', 'end

In [14]:
# load mappluto
#pluto = gpd.read_file("data/nyc_mappluto_25v1_1_fgdb/MapPLUTO25v1_1.gdb")
#plutocb3 = pluto[pluto['CD']==103]
#plutocb3.to_file('data/cb3_pluto.geojson')
plutocb3 = gpd.read_file('data/cb3_pluto.geojson')

In [15]:
plutocb3_subhousing = plutocb3.merge(df_subsidized.rename(columns={'bbl':'BBL'}), on='BBL')

In [25]:
nycha = plutocb3_subhousing[(plutocb3_subhousing['prog_nycha_mix']==1)|(plutocb3_subhousing['prog_nycha_ph']==1)]
nycha.to_crs(4326).to_file('data/nycha.geojson')

In [26]:
sec8 = plutocb3_subhousing[(plutocb3_subhousing['prog_proj8']==1)|(plutocb3_subhousing['prog_rad']==1)]
sec8.to_crs(4326).to_file('data/sec8.geojson')

In [30]:
ml = plutocb3_subhousing[(plutocb3_subhousing['prog_ml']==1)]
ml.to_crs(4326).to_file('data/ml.geojson')

In [74]:
knickerbocker_village = plutocb3[plutocb3['BBL'].astype(str).str.contains('002530001')][['Address', 'OwnerName', 'geometry']]
knickerbocker_village.to_crs(4326).to_file('data/knickerbocker_village.geojson')

## Load Crashes

In [149]:
# load 2023 sectors
gdf_sectors = gpd.read_file("data/acs_2023_mapbox.geojson")

In [150]:
#df_crash = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=10000000")
#df_crash = gpd.read_file("https://data.cityofnewyork.us/resource/h9gi-nx95.geojson?$query=SELECT%0A%20%20%60crash_date%60%2C%0A%20%20%60crash_time%60%2C%0A%20%20%60borough%60%2C%0A%20%20%60zip_code%60%2C%0A%20%20%60latitude%60%2C%0A%20%20%60longitude%60%2C%0A%20%20%60location%60%2C%0A%20%20%60on_street_name%60%2C%0A%20%20%60off_street_name%60%2C%0A%20%20%60cross_street_name%60%2C%0A%20%20%60number_of_persons_injured%60%2C%0A%20%20%60number_of_persons_killed%60%2C%0A%20%20%60number_of_pedestrians_injured%60%2C%0A%20%20%60number_of_pedestrians_killed%60%2C%0A%20%20%60number_of_cyclist_injured%60%2C%0A%20%20%60number_of_cyclist_killed%60%2C%0A%20%20%60number_of_motorist_injured%60%2C%0A%20%20%60number_of_motorist_killed%60%2C%0A%20%20%60contributing_factor_vehicle_1%60%2C%0A%20%20%60contributing_factor_vehicle_2%60%2C%0A%20%20%60contributing_factor_vehicle_3%60%2C%0A%20%20%60contributing_factor_vehicle_4%60%2C%0A%20%20%60contributing_factor_vehicle_5%60%2C%0A%20%20%60collision_id%60%2C%0A%20%20%60vehicle_type_code1%60%2C%0A%20%20%60vehicle_type_code2%60%2C%0A%20%20%60vehicle_type_code_3%60%2C%0A%20%20%60vehicle_type_code_4%60%2C%0A%20%20%60vehicle_type_code_5%60%0AWHERE%0A%20%20%60crash_date%60%0A%20%20%20%20BETWEEN%20%222024-06-02T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20AND%20%222025-06-02T23%3A22%3A11%22%20%3A%3A%20floating_timestamp")
df_crash = pd.read_csv("data/Motor_Vehicle_Collisions_-_Crashes_20250602.csv")
gdf_crash = gpd.GeoDataFrame(
    df_crash, geometry=gpd.points_from_xy(df_crash.LONGITUDE, df_crash.LATITUDE), crs="EPSG:4326"
)
gdf_crash.columns = gdf_crash.columns.str.lower().str.replace(' ', '_')

In [151]:
gdf_crash_cb3 = gdf_crash.sjoin(gdf_sectors, how='inner')

In [152]:
gdf_crash_cb3['crash_date'] = pd.to_datetime(gdf_crash_cb3['crash_date'])
gdf_crash_cb3['crash_month'] = gdf_crash_cb3['crash_date'].dt.month
gdf_crash_cb3['crash_year'] = gdf_crash_cb3['crash_date'].dt.year

In [153]:
gdf_crash_cb3_summary = gdf_crash_cb3.groupby(['sectors', 'crash_month', 'crash_year'], as_index=False)[['number_of_motorist_injured', 'number_of_cyclist_killed', 
'number_of_cyclist_injured', 'number_of_persons_killed', 'number_of_persons_injured',
'number_of_pedestrians_killed', 'number_of_pedestrians_injured']].sum().fillna(0)
gdf_crash_cb3_summary

,sectors,crash_month,crash_year,number_of_motorist_injured,number_of_cyclist_killed,number_of_cyclist_injured,number_of_persons_killed,number_of_persons_injured,number_of_pedestrians_killed,number_of_pedestrians_injured
0,1,1,2025,1,0,1,0,4,0,2
1,1,2,2025,1,0,1,0,4,0,2
2,1,3,2025,2,0,1,0,3,0,0
3,1,4,2025,2,0,2,0,6,0,2
4,1,5,2025,1,0,4,0,12,0,6
...,...,...,...,...,...,...,...,...,...,...
115,9,8,2024,3,0,3,0,11,0,4
116,9,9,2024,4,0,1,0,8,0,3
117,9,10,2024,5,0,3,0,14,0,6
118,9,11,2024,2,0,1,0,7,0,4


In [154]:
gdf_crash_cb3_summary = gdf_crash_cb3_summary.groupby('sectors')[['number_of_motorist_injured', 'number_of_cyclist_killed', 
'number_of_cyclist_injured', 'number_of_persons_killed', 'number_of_persons_injured',
'number_of_pedestrians_killed', 'number_of_pedestrians_injured']].mean()

In [155]:
gdf_sectors.merge(gdf_crash_cb3_summary, on='sectors').to_file("data/acs_2023_mapbox.geojson")